In [7]:
import json
import pandas as pd
import requests
import os
import xmltodict
from tqdm import tqdm

In [8]:
filepath = 'data/'
filelist = os.listdir(filepath)
# filelist
conf = open('config.json','r') 
config = json.load(conf)
api_key = config['API_key']
conf.close()
headers=  {'X-ELS-APIKey':api_key}

In [9]:
filelist

['waste reduction.csv',
 'renewable energy.csv',
 'wastewater treatment.csv',
 'additive manufacturing.csv',
 'geothermal.csv',
 'green building.csv',
 'carbon emissions.csv',
 'heat recovery.csv',
 'smart home.csv',
 'zero waste.csv',
 'maritime electrification.csv',
 'green energy.csv',
 'energy storage.csv',
 'clean technology.csv',
 'clean-energy.csv',
 'energy efficient.csv']

In [11]:
for filename in filelist:
    keyword = filename.replace(".csv","")
    raw_data = pd.read_csv(filepath + filename,header=0)
    raw_data["abstract"] = None
    raw_data["topic"] = keyword
    print(keyword)
    for i in tqdm(range(0,len(raw_data))):
        doi = raw_data['doi'][i]
        aburl = 'https://api.elsevier.com/content/article/doi/'+doi
        # print(aburl)
        abstract = requests.get(aburl,headers = headers)
        try:
            d = xmltodict.parse(abstract.text)
            abstract_text = d["full-text-retrieval-response"]['coredata']['dc:description']
            raw_data.at[i,'abstract'] = abstract_text
            pass
        except:
            continue
    select_data = raw_data.dropna()
    select_data.to_csv(filepath + keyword + '-selected.csv', encoding='utf-8',index=False)
    

waste reduction


  9%|▉         | 431/4854 [04:39<47:46,  1.54it/s]  


KeyboardInterrupt: 

In [13]:
import multiprocessing
def getabstract(filename):
    keyword = filename.replace(".csv","")
    raw_data = pd.read_csv(filepath + filename,header=0)
    raw_data["abstract"] = None
    raw_data["topic"] = keyword
    print(keyword)
    for i in tqdm(range(0,len(raw_data))):
        doi = raw_data['doi'][i]
        aburl = 'https://api.elsevier.com/content/article/doi/'+doi
        # print(aburl)
        abstract = requests.get(aburl,headers = headers)
        try:
            d = xmltodict.parse(abstract.text)
            abstract_text = d["full-text-retrieval-response"]['coredata']['dc:description']
            raw_data.at[i,'abstract'] = abstract_text
            pass
        except:
            continue
    select_data = raw_data.dropna()
    select_data.to_csv(filepath + keyword + '-selected.csv', encoding='utf-8',index=False)

pool_obj = multiprocessing.Pool(4)
pool_obj.map(getabstract,filelist)
    



Process SpawnPoolWorker-11:
Process SpawnPoolWorker-12:
Process SpawnPoolWorker-13:
Process SpawnPoolWorker-15:
Process SpawnPoolWorker-14:
Process SpawnPoolWorker-16:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'getabstract' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File

KeyboardInterrupt: 